#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 5, 15)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

In [3]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,20,S,SYNEX,"3,000",28.25,0,CP1S,SET100,2022-03-10


In [5]:
tmp = pd.read_csv('../data/orders-log.csv')
tmp['trans'] = tmp.apply(lambda row: categorise(row), axis=1)
tmp['amount'] = tmp.qty * tmp.target
tmp[tmp.active == 2].style.background_gradient(cmap = 'Blues')

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount


In [6]:
tmp.head().sort_values('spd',ascending=False).style.hide_index()

trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
B,DIF,-1,DOS,SET,10000,14.200000,14.300000,0.000000,0.00%,0,2022-05-17,Buy,142000.000000
B,BBL,-2,1L,SET50,2400,124.000000,125.000000,-1.000000,-0.79%,0,2022-04-22,Buy,297600.000000
B,HREIT,-3,DOS,SET,10000,8.200000,8.350000,0.050000,+0.60%,0,2022-03-04,Buy,82000.000000
B,BDMS,-4,1L,SET50,4200,26.000000,27.000000,1.000000,+3.85%,0,2022-03-09,Buy,109200.000000
B,BGC,-4,DOS,SET,10000,9.800000,10.000000,0.200000,+2.04%,0,2022-05-25,Buy,98000.000000


### End of Tables in the process

### Print to verify before upload file

In [30]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [31]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [32]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,BDMS,"4,200",26.00,"109,200.00",1L,SET50,2,2022-03-09
1,B,BGC,"10,000",9.80,"98,000.00",DOS,SET,2,2022-05-25
2,B,IVL,"2,400",45.00,"108,000.00",XXX,SET50,2,2022-05-27
3,B,WHART,"10,000",10.50,"105,000.00",RD10pct,SET,2,2022-06-01
4,S,PTT,"3,000",37.00,"111,000.00",C38.50,SET50,2,2022-03-03
5,S,RATCH,"2,000",38.50,"77,000.00",CUT,SET50,2,2022-03-17


In [33]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    420200.0
S    188000.0
Name: amount, dtype: float64

In [34]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,BDMS,"4,200",26.00,"109,200.00",1L,SET50,2,2022-03-09
1,B,BGC,"10,000",9.80,"98,000.00",DOS,SET,2,2022-05-25
2,B,IVL,"2,400",45.00,"108,000.00",XXX,SET50,2,2022-05-27
3,B,WHART,"10,000",10.50,"105,000.00",RD10pct,SET,2,2022-06-01


In [35]:
cash = 312_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(312000, 420200.0, -108200.0)

In [36]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
4,S,PTT,"3,000",37.00,"111,000.00",C38.50,SET50,2,2022-03-03
5,S,RATCH,"2,000",38.50,"77,000.00",CUT,SET50,2,2022-03-17


In [37]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(312000, 188000.0, 500000.0)

### After call ord-log (must call everytime that orders change)

In [38]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [39]:
show(df[['trans','name','spd','reason','qty','target','current','active']])

trans,name,spd,reason,qty,target,current,active


In [40]:
mb = (df.trans == 'Buy') & (df.spd >= -3)
df[mb].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,BBL,-2,1L,SET50,2400,124.0,125.00,-1.00,-0.79%,1,2022-04-22,Buy,297600.0
3,B,DIF,-1,DOS,SET,10000,14.2,14.30,0.00,0.00%,1,2022-05-17,Buy,142000.0
4,B,HREIT,-3,DOS,SET,10000,8.2,8.35,0.05,+0.60%,1,2022-03-04,Buy,82000.0
6,B,JASIF,-2,DOS,SET,10000,10.8,11.00,0.10,+0.92%,1,2022-05-24,Buy,108000.0
9,B,WHART,-1,RD10pct,SET,10000,10.5,10.60,0.10,+0.95%,2,2022-06-01,Buy,105000.0


In [41]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [42]:
ms = (df.trans == 'Sell') & (df.spd <= 5)
df[ms].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
15,S,PTT,3,C38.50,SET50,3000,37.0,36.25,-0.25,-0.68%,2,2022-03-03,Sell,111000.0
17,S,RATCH,1,CUT,SET50,2000,38.5,38.25,-1.50,-3.77%,2,2022-03-17,Sell,77000.0


In [43]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,BBL,-2,1L,SET50,2400,124.0,125.00,-1.00,-0.79%,1,2022-04-22,Buy,297600.0
3,B,DIF,-1,DOS,SET,10000,14.2,14.30,0.00,0.00%,1,2022-05-17,Buy,142000.0
4,B,HREIT,-3,DOS,SET,10000,8.2,8.35,0.05,+0.60%,1,2022-03-04,Buy,82000.0
6,B,JASIF,-2,DOS,SET,10000,10.8,11.00,0.10,+0.92%,1,2022-05-24,Buy,108000.0
9,B,WHART,-1,RD10pct,SET,10000,10.5,10.60,0.10,+0.95%,2,2022-06-01,Buy,105000.0
15,S,PTT,3,C38.50,SET50,3000,37.0,36.25,-0.25,-0.68%,2,2022-03-03,Sell,111000.0
17,S,RATCH,1,CUT,SET50,2000,38.5,38.25,-1.50,-3.77%,2,2022-03-17,Sell,77000.0


In [44]:
df[mb | ms].shape[0]

7

In [45]:
mask = (df.trade == 'B')
df[mask].nlargest(5, "amount")[["name","trans","amount","spd"]].style.format(format_dict)

,name,trans,amount,spd
0,BBL,Buy,"297,600.00",-2
7,KBANK,Buy,"210,000.00",-4
8,KKP,Buy,"204,000.00",-5
3,DIF,Buy,"142,000.00",-1
1,BDMS,Buy,"109,200.00",-4


### Select source of orders between these two

In [46]:
df_out = df[mb | ms][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
0,Buy,BBL,2400,124.0,1,-2,125.00,-1.00,-0.79%,1L,SET50,2022-04-22
3,Buy,DIF,10000,14.2,1,-1,14.30,0.00,0.00%,DOS,SET,2022-05-17
4,Buy,HREIT,10000,8.2,1,-3,8.35,0.05,+0.60%,DOS,SET,2022-03-04
6,Buy,JASIF,10000,10.8,1,-2,11.00,0.10,+0.92%,DOS,SET,2022-05-24
9,Buy,WHART,10000,10.5,2,-1,10.60,0.10,+0.95%,RD10pct,SET,2022-06-01
15,Sell,PTT,3000,37.0,2,3,36.25,-0.25,-0.68%,C38.50,SET50,2022-03-03
17,Sell,RATCH,2000,38.5,2,1,38.25,-1.50,-3.77%,CUT,SET50,2022-03-17


In [47]:
df_out = df[df.active == 2][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
1,Buy,BDMS,4200,26.0,2,-4,27.00,1.00,+3.85%,1L,SET50,2022-03-09
2,Buy,BGC,10000,9.8,2,-4,10.00,0.20,+2.04%,DOS,SET,2022-05-25
5,Buy,IVL,2400,45.0,2,-6,46.50,2.25,+5.08%,XXX,SET50,2022-05-27
9,Buy,WHART,10000,10.5,2,-1,10.60,0.10,+0.95%,RD10pct,SET,2022-06-01
15,Sell,PTT,3000,37.0,2,3,36.25,-0.25,-0.68%,C38.50,SET50,2022-03-03
17,Sell,RATCH,2000,38.5,2,1,38.25,-1.50,-3.77%,CUT,SET50,2022-03-17


In [48]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process